# Tutorial 4: Improving LLMs with RLHF

Reinforcement Learning from Human Feedback (RLHF) incorporates human feedback into the training process through a reward model that learns the desired patterns to improve the model’s output. For example, if the goal is to enhance politeness, the reward model will guide the model to generate more polite responses by assigning higher scores to polite outputs. This process is resource-intensive because it necessitates training a reward model using a dataset curated by humans.

This tutorial will use available open-source models and datasets whenever possible while maintaining costs.

We begin with a pre-trained model that we fine-tune in a supervised fine-tuning phase using the `SFTTrainer` class. Next, a reward model is trained with the desired traits using the `RewardTrainer` class. Finally, the reinforcement learning phase employs the models to build the ultimate aligned model, utilizing the `PPOTrainer`.

You can access the reports generated from Weights & Biases and the file with the requirements for the library after each subsection. Note that different steps require distinct versions of libraries. We chose `OPT-1.3B` as the base model and fine-tuned a `DeBERTa` (300M) model as the reward model for our experiments. While these are more compact models, the process used in this tutorial can be applied to other existing models by simply modifying the model’s name in the code.

Even if much more affordable than what companies like OpenAI do, this tutorial is still resource-intensive as we replicate an RLHF phase. We rented an 8x NVIDIA A100 instance for $8.80/h and used  [lambda](https://lambdalabs.com/)  as our GPU cloud provider.

>⚠️It’s important to be aware of the costs associated with cloud GPUs. The total cost will depend on the machine type and the instance’s uptime. Regularly check your costs in the billing section of Lambda Labs and spin off your instances when you don’t use them.

>💡If you want to run the code in the section without spending much money, you can perform a few iterations of training on your virtual machine and then stop it.

## Supervised Fine-Tuning

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20FineTuning_a_LLM_QLoRA.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

Previous sections covered the SFT phase. This section uses a unique  [OpenOrca](https://huggingface.co/datasets/Open-Orca/OpenOrca)  dataset with question-response pairs and implements the QLoRA fine-tuning technique.

This phase teaches the model a conversational format, training it to provide answers rather than defaulting to its standard auto-completion function.

Install the required libraries with the command `!pip install -q transformers==4.32.0 bitsandbytes==0.41.1 accelerate==0.22.0 deeplake==3.6.19 trl==0.5.0 peft==0.5.0 wandb==0.15.8.`

### The Dataset

The first step is streaming the dataset. Streaming a dataset refers to the process of loading and processing data in smaller chunks or batches rather than loading the entire dataset into memory at once. This approach is useful when working with large datasets that cannot fit into memory or when you want to perform real-time processing. For this example, we only use a subset of the original dataset, comprising 1 million data points. However, you can access the  [entire dataset](https://app.activeloop.ai/genai360/OpenOrca-4M/), containing 4 million data points, at  [towardsai.net/book](http://towardsai.net/book).

In [ ]:
import deeplake

# Connect to the training and testing datasets
ds = deeplake.load('hub://genai360/OpenOrca-1M-train-set')
ds_valid = deeplake.load('hub://genai360/OpenOrca-1M-valid-set')

print(ds)

    Dataset(path='hub://genai360/OpenOrca-1M-train-set',  
    read_only=True,  
    tensors=['id', 'question', 'response', 'system_prompt'])

The dataset features three key columns: question, the queries posed to the LLM; response, the model’s output or answers to these questions; and `system_prompt`, the initial instructions that set the context for the model, such as “you are a helpful assistant.”

For simplicity, this chapter focuses solely on the first two columns. However, incorporating system prompts into text formatting can be advantageous. The text is structured in the format `Question: xxx\n\nAnswer: yyy`, with the question and answer separated by two newline characters. You can also experiment with different formats, such as `System: xxx\n\nQuestion: yyy\n\nAnswer: zzz`, to include the system prompts from the dataset.

In [ ]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"""Question: {example['question'][0]}\n\nAnswer: {example['response'][0]}"""
    return text

Next, load the OPT model tokenizer:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

Use the `ConstantLengthDataset` class to aggregate data. This will maximize usage within the 2K input size constraint and improve training efficiency.

In [ ]:
from trl.trainer import ConstantLengthDataset

train_dataset = ConstantLengthDataset(
    tokenizer,
    ds,
    formatting_func=prepare_sample_text,
    infinite=True,
    seq_length=2048
)

eval_dataset = ConstantLengthDataset(
    tokenizer,
    ds_valid,
    formatting_func=prepare_sample_text,
    seq_length=1024
)

iterator = iter(train_dataset)
sample = next(iterator)
print(sample)

train_dataset.start_iteration = 0

    {'input_ids': tensor([ 16, 358, 828, ..., 137, 79, 362]),  
    'labels': tensor([ 16, 358, 828, ..., 137, 79, 362])}